In [1]:
import csv
import numpy as np
import os
import pandas as pd
import requests
import time

In [2]:
####### herb compound의 stitch ID를 mapping하는 작업. pubchem ID를 매개로 사용할 것이다.

In [4]:
sider_cid = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_0_stitch(sider)_pubchem.tsv")
herb_cid = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/2-5_herb_pubchem.tsv")

In [5]:
print len(sider_cid)
sider_cid.head(1)

13265


,stitch_flat,stitch_stereo,pubchem_cid
0,CID100216416,CID000216416,10671241


In [6]:
print len(herb_cid)
herb_cid.head(1)

11395


,compID,compName,pubchemID
0,32469,"1,3,11-Elematriene",10583


In [7]:
a = sider_cid[sider_cid['pubchem_cid'].isin(herb_cid['pubchemID'])]
print len(a)
print len(a.drop_duplicates(subset = ['stitch_flat', 'stitch_stereo']))
a.head(2)

124
94


,stitch_flat,stitch_stereo,pubchem_cid
179,CID100000137,CID000000137,137
297,CID100000774,CID000000774,25201573


In [8]:
b = herb_cid[herb_cid['pubchemID'].isin(sider_cid['pubchem_cid'])]
print len(b)
print len(b.drop_duplicates(subset = 'compID'))
b.head(2)

153
113


,compID,compName,pubchemID
4,5525,Benzyl alcohol,244
93,1020,Probenecid,4911


In [9]:
result = pd.DataFrame()
for index, row in b.iterrows():
    pubchem_id = row['pubchemID']
    filtered = sider_cid[sider_cid.pubchem_cid == pubchem_id]
    if filtered.empty:
        continue
    else:
        for index2, row2 in filtered.iterrows():
            flat = row2['stitch_flat']
            stereo = row2['stitch_stereo']
            temp_df = pd.DataFrame([[row['compID'], row['compName'], flat, stereo]], 
                                  columns = ['compID', 'compName', 'stitch_flat', 'stitch_stereo'])
            result = result.append(temp_df)

In [10]:
print len(result)
print len(result.drop_duplicates())

153
113


In [11]:
result.head()

,compID,compName,stitch_flat,stitch_stereo
0,5525,Benzyl alcohol,CID100000244,CID000000244
0,1020,Probenecid,CID100004911,CID000004911
0,732,D-(-)-Mannitol,CID100000453,CID000006251
0,9210,Acetic acid,CID100000175,CID000000175
0,926,Salicylic acid,CID100000338,CID000000338


In [9]:
result.drop_duplicates().to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_1_herb_stitchID(SIDER).tsv", sep='\t', index=False)
result.drop_duplicates().to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/6_1_herb_stitchID(SIDER).tsv", sep='\t', index=False)

In [12]:
######## herb compound의 바꾼 stitch id를 사용. SIDER에서 해당하는 side effect를 뽑아낸다.

In [13]:
raw_sider = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/meddra_all_label_se_header.tsv")

In [14]:
raw_sider_pt = raw_sider[raw_sider.MedDRA_concept_type == 'PT']
print(len(raw_sider_pt))
raw_sider_pt.head(1)

2523626


,source_label,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id,MedDRA_concept_type,UMLS_concept_id_for_MedDRA,side_effect_name
1,EMA/WC500020092.html,CID100216416,CID000216416,C0000737,PT,C0000737,Abdominal pain


In [15]:
sider = raw_sider_pt[['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'UMLS_concept_id_for_MedDRA', 'side_effect_name']]

In [16]:
sider_wo_dup = sider.drop_duplicates()
print(len(sider_wo_dup))
sider_wo_dup.head()

152759


,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id_for_MedDRA,side_effect_name
1,CID100216416,CID000216416,C0000737,Abdominal pain
2,CID100216416,CID000216416,C0687713,Gastrointestinal pain
4,CID100216416,CID000216416,C0002170,Alopecia
6,CID100216416,CID000216416,C0002395,Dementia Alzheimer's type
8,CID100216416,CID000216416,C0002622,Amnesia


In [17]:
sider_result = pd.DataFrame()
for index, row in result.iterrows():
    flat = row['stitch_flat']
    stereo = row['stitch_stereo']
    filtered = sider_wo_dup[(sider_wo_dup.STITCH_compound_id_flat == flat) & (sider_wo_dup.STITCH_compound_id_stereo == stereo)]
    if filtered.empty:
        print flat, stereo
        continue
    else:
        for index2, row2 in filtered.iterrows():
            umls_id = row2['UMLS_concept_id_for_MedDRA']
            side_effect = row2['side_effect_name']
            temp_df = pd.DataFrame([[row['compID'], row['compName'], umls_id, side_effect]], 
                                   columns = ['compID', 'compName', 'umls_id', 'side_effect_name'])
            sider_result = sider_result.append(temp_df)

In [18]:
print len(sider_result)
print len(sider_result.drop_duplicates())

8525
7885


In [19]:
sider[sider.duplicated(keep=False)].sort_values(by = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo','UMLS_concept_id_for_MedDRA']).head()

,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id_for_MedDRA,side_effect_name
1331580,CID100000085,CID000010917,C0000737,Abdominal pain
1376376,CID100000085,CID000010917,C0000737,Abdominal pain
1460376,CID100000085,CID000010917,C0000737,Abdominal pain
3391903,CID100000085,CID000010917,C0000737,Abdominal pain
3453661,CID100000085,CID000010917,C0000737,Abdominal pain


In [20]:
sider_result = sider_result.drop_duplicates()

In [21]:
print len(sider_result.drop_duplicates(subset = 'compID')) # SIDER의 side effect를 가지고 있는 herb compound의 수
print len(sider_result.drop_duplicates(subset = 'umls_id')) # SIDER의 side effect의 전체 수
sider_result.head()

113
1406


,compID,compName,umls_id,side_effect_name
0,5525,Benzyl alcohol,C0013604,Oedema
0,5525,Benzyl alcohol,C0011603,Dermatitis
0,5525,Benzyl alcohol,C0015230,Rash
0,5525,Benzyl alcohol,C0015256,Excoriation
0,5525,Benzyl alcohol,C0020580,Hypoaesthesia


In [23]:
sider_result.to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_2_herb_sider_sideEffect.tsv", sep='\t', index=False)
sider_result[['compID', 'compName', 'umls_id']].to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_3_herb_sider_umlsID.tsv", sep='\t', index=False)
sider_result.to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/6_2_herb_sider_sideEffect.tsv", sep='\t', index=False)
sider_result[['compID', 'compName', 'umls_id']].to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/6_3_herb_sider_umlsID.tsv", sep='\t', index=False)

In [ ]:
########## 여기서부터는 OFFSIDE

In [22]:
stitch_cid = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv")

In [23]:
print len(stitch_cid)
stitch_cid.head(2)

47651221


,flat_chemical,stereo_chemical,source_name,source_id
0,CID100000001,CID000000001,PC,1
1,CID100000010,CID000000010,PC,10


In [24]:
raw_offisde = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3003377s-offsides.tsv")

In [25]:
print(len(raw_offisde))
offside = raw_offisde[['stitch_id','drug', 'umls_id', 'event']]
offside.head(2)

438801


,stitch_id,drug,umls_id,event
0,CID000000076,dehydroepiandrosterone,C0000737,abdominal pain
1,CID000000076,dehydroepiandrosterone,C0001622,hyperadrenalism


In [26]:
g = stitch_cid[stitch_cid.stereo_chemical.isin(raw_offisde['stitch_id'])]
print len(g)
print len(g.drop_duplicates(subset = 'flat_chemical')) # 여기서 왜 flat chemical을 하는지 모름...
g.head(1)

8878
1255


,flat_chemical,stereo_chemical,source_name,source_id
5561,CID100005073,CID000005073,PC,10005067


In [46]:
# OFFSIDE의 stitch id과 pubchem id를 매칭해놓은것.
g.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/4_0_stitch(offside)_pubchem.tsv")
g.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/4_0_stitch(offside)_pubchem.tsv")

In [27]:
print len(g[g.source_id.isin(herb_cid['pubchemID'])]) 
print len(g[g.source_id.isin(herb_cid['pubchemID'])].drop_duplicates(subset = 'stereo_chemical')) # offside 정보를 가지고 있는 herb compound
g[g.source_id.isin(herb_cid['pubchemID'])].head(2)

59
52


,flat_chemical,stereo_chemical,source_name,source_id
347276,CID100001032,CID000001032,PC,1032
515854,CID100001032,CID000001032,PC,104745


In [28]:
h = herb_cid[herb_cid['pubchemID'].isin(g['source_id'])] 
print len(h)
print len(h.drop_duplicates(subset = 'compID'))# herb compound 중, offisde 정보를 가지고 있는 herb compound의 수 (위아래가 왜 수가 다른지는 모를..)
h.head(2)

61
61


,compID,compName,pubchemID
93,1020,Probenecid,4911
220,926,Salicylic acid,338


In [29]:
# herb copmound와 offside id를 매핑하는 과정 (pubchem id 사용)
result2 = pd.DataFrame()
for index, row in h.iterrows():
    pubchem_id = row['pubchemID']
    filtered = g[g.source_id == pubchem_id]
    if filtered.empty:
        continue
    else:
        for index2, row2 in filtered.iterrows():
            stereo = row2['stereo_chemical']
            temp_df = pd.DataFrame([[row['compID'], row['compName'], stereo]], 
                                  columns = ['compID', 'compName','stitch_stereo'])
            result2 = result2.append(temp_df)

In [30]:
# 위에서 바꾼 offside id를 통해 herb compound마다 adr을 찾는다. 
offside_result = pd.DataFrame()
for index, row in result2.iterrows():
    stereo = row['stitch_stereo']
    filtered = raw_offisde[raw_offisde.stitch_id == stereo]
    if filtered.empty:
        print stereo
        continue
    else:
        for index2, row2 in filtered.iterrows():
            umls_id = row2['umls_id']
            side_effect = row2['event']
            temp_df = pd.DataFrame([[row['compID'], row['compName'], umls_id, side_effect]], 
                                   columns = ['compID', 'compName', 'umls_id', 'side_effect_name'])
            offside_result = offside_result.append(temp_df)

In [31]:
print len(offside_result)
print len(offside_result.drop_duplicates())
print len(offside_result.drop_duplicates(subset = 'compID')) # offside의 정보를 갖고 있는 herb compound의 수
print len(offside_result.drop_duplicates(subset ='umls_id')) # 그것의 adr의 수
offside_result.head(2)

21487
21487
61
4516


,compID,compName,umls_id,side_effect_name
0,1020,Probenecid,C0000737,abdominal pain
0,1020,Probenecid,C0001122,Acidosis


In [73]:
offside_result.to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_2_herb_offside_sideEffect.tsv", sep = '\t', index = False)
offside_result[['compID', 'compName', 'umls_id']].to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_3_herb_offside_umlsID.tsv", sep = '\t', index = False)
offside_result.to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/6_2_herb_offside_sideEffect.tsv", sep = '\t', index = False)
offside_result[['compID', 'compName', 'umls_id']].to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/6_3_herb_offside_umlsID.tsv", sep = '\t', index = False)

In [ ]:
# herb_compound가 갖고 있는 side effect feature와 gold drug의 common feature 비교해서 걸러내기

In [32]:
drug = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/8_gold_standard_drug_binary_matrix.tsv", index_col = 0)

In [35]:
common_feature = pd.Series(drug.index.values)
print len(common_feature)

2276


In [34]:
len(sider_result[sider_result['umls_id'].isin(common_feature)].drop_duplicates(subset = 'umls_id'))

947

In [36]:
len(offside_result[offside_result['umls_id'].isin(common_feature)].drop_duplicates(subset = 'umls_id'))

1765

In [ ]:
# 기존 COCONUT database에서 제공하는 정보도 합쳐야함.

In [37]:
herb_cause = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/5_2_herb_compound_phenotype_cause_umls.tsv")

In [38]:
len(herb_cause[herb_cause['compID'].isin(sider_result['compID'])].drop_duplicates(subset = 'compID'))

35

In [40]:
coconut = herb_cause[['compID', 'umlsID']]
coconut = coconut.rename(columns = {'umlsID' : 'umls_id'})
sider = sider_result[['compID', 'umls_id']] 
offside = offside_result[['compID', 'umls_id']]
frames = [coconut, sider, offside]
combined = pd.concat(frames)

In [41]:
combined.head(1)

,compID,umls_id
0,1020.0,C0002170


In [42]:
print len(combined.drop_duplicates(subset = 'compID')) # herb compound 의 수
print len(combined.drop_duplicates(subset = 'umls_id')) # 그 herb compound가 가지고 있는 side effect수

143
5413


In [46]:
print len(combined[combined['umls_id'].isin(common_feature)].drop_duplicates(subset = 'compID')) # gold standard drug feature를 가지고 있는 herb compound의 수
print len(combined[combined['umls_id'].isin(common_feature)].drop_duplicates(subset = 'umls_id')) # gold standard drug feature와 겹치는 feature의 수
k = combined[combined['umls_id'].isin(common_feature)]

143
1879


In [50]:
k.head(2)

,compID,umls_id
0,1020.0,C0002170
1,1020.0,C0018965


In [52]:
k.to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/7_herb_compound_combined_common_phenotypic_effects.tsv", sep='\t', index=False)
k.to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/7_herb_compound_combined_common_phenotypic_effects.tsv", sep='\t', index=False)

In [ ]:
### learning이 가능한 herb compound(바로 위에서 구한 k)와 disease의 association

In [44]:
herb_asso = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/5_2_herb_compound_phenotype_association_umls.tsv")

In [45]:
herb_asso.head(1)

,compID,effectType,evidence,phenID,reference,umlsID
0,32837.0,Association,Central Nervous System Diseases,2304.0,CTD[C011015],C0007682


In [47]:
print len(herb_asso[herb_asso['compID'].isin(k['compID'])].drop_duplicates())  # row의 수가 learning이 가능한 herb compound와 disease의 association 수
herb_asso[herb_asso['compID'].isin(k['compID'])].drop_duplicates().head()

6037


,compID,effectType,evidence,phenID,reference,umlsID
349,5525.0,Association,Adenocarcinoma,912.0,CTD[D019905],C0001418
350,5525.0,Association,Alcoholism,312.0,CTD[D019905],C0001973
351,5525.0,Association,"Arrhythmias, Cardiac",479.0,CTD[D019905],C0003811
352,5525.0,Association,Breast Neoplasms,3351.0,CTD[D019905],C1458155
353,5525.0,Association,"Carcinoma, Hepatocellular",1131.0,CTD[D019905],C2239176


In [53]:
learning_herb_asso = herb_asso[herb_asso['compID'].isin(k['compID'])].drop_duplicates()
learning_herb_asso.to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/7_2_learning_herb_compound_disease_association.tsv", sep='\t', index=False)
learning_herb_asso.to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/7_2_learning_herb_compound_disease_association.tsv", sep='\t', index=False)

In [49]:
# 이건 coconut에 있는 feature로만 찾은 herb compound -disease association임
print len(herb_asso[herb_asso['compID'].isin(herb_cause['compID'])].drop_duplicates())
herb_asso[herb_asso['compID'].isin(herb_cause['compID'])].drop_duplicates()

4900


,compID,effectType,evidence,phenID,reference,umlsID
2578,1020.0,Association,Cytomegalovirus Retinitis,1042.0,ClinicalTrial[NCT00304434] / ClinicalTrial[NCT...,C0206178
2579,1020.0,Association,HIV Infections,109.0,ClinicalTrial[NCT00304434] / ClinicalTrial[NCT...,C0019693
2580,1020.0,Association,Acute Kidney Injury,6088.0,CTD[D011339],C2609414
2581,1020.0,Association,AIDSRelated Opportunistic Infections,4594.0,CTD[D011339],C0029118
2582,1020.0,Association,Albuminuria,3638.0,CTD[D011339],C0001925
2583,1020.0,Association,Allanson Pantzar McLeod syndrome,10484.0,CTD[D011339],C0266313
2584,1020.0,Association,AmphetamineRelated Disorders,184.0,CTD[D011339],C0012634
2585,1020.0,Association,"Anemia, Hemolytic, Autoimmune",1999.0,CTD[D011339],C0002880
2586,1020.0,Association,Anxiety Disorders,459.0,CTD[D011339],C0003469
2587,1020.0,Association,"Arthritis, Gouty",2388.0,CTD[D011339],C0003868


In [ ]:
추가 작업 필요함 -> association 이 있다고 하는 disease의 feature를 구하고(DisNet에서)
이 feature와 gold standard disease feature와 공통을 구해서 사용가능한 association을 filter해야함.
